In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_blobs
import torch.nn.functional as F
from torch import nn
from torch import optim

In [ ]:
class NumpyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = F.one_hot(torch.tensor(y, dtype=torch.int64)).type(torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        input = self.X[idx, :]
        output = self.y[idx, :]

        return input, output

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_size, out_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_size, 16)
        self.do1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(16, 16)
        self.do2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(16, out_size)

    def forward(self, x):
        a1 = F.relu(self.fc1(x))
        a1 = self.do1(a1)
        a2 = F.relu(self.fc2(a1))
        a2 = self.do2(a2)
        out = F.softmax(self.fc3(a2), dim=1)

        return out

In [ ]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    model.eval()
    with torch.no_grad():
        bar = tqdm(loader)
        for batch, (x, y) in enumerate(bar):
            x = x.to(device=device)
            y = torch.argmax(y, dim=1).to(device=device)

            probs = model(x)
            _, y_pred = probs.max(1)

            num_correct += (y_pred == y).sum()
            num_samples += y_pred.size(0)

    return f"{(num_correct / num_samples) * 100}%"

In [ ]:
n_samples = 100
n_features = 5
n_classes = 3
batch_size = 8
lr = 1e-2
epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X, y = make_blobs(n_samples=n_samples, n_features=n_features, centers=n_classes)
train_size = int(0.8 * n_samples)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

train_dataset = NumpyDataset(X_train, y_train)
test_dataset = NumpyDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

mlp = MLP(n_features, n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(mlp.parameters(), lr=lr)

mlp.train()
for ep in range(epochs):
    bar = tqdm(train_dataloader)
    for batch, (X, y) in enumerate(bar):
        X = X.to(device)
        y = y.to(device)

        probs = mlp(X)
        loss = criterion(probs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        bar.set_description(f"<EP>{ep+1}")
        bar.set_postfix(loss=loss)

<EP>10: 100%|██████████| 10/10 [00:00<00:00, 117.70it/s, loss=tensor(0.5514, grad_fn=<DivBackward1>)]


In [ ]:
check_accuracy(train_dataloader, mlp)

100%|██████████| 10/10 [00:00<00:00, 917.49it/s]


'100.0%'

In [ ]:
check_accuracy(test_dataloader, mlp)

100%|██████████| 3/3 [00:00<00:00, 1040.68it/s]


'100.0%'